#DATASCI W261: Machine Learning at Scale

**Nick Hamlin** (nickhamlin@gmail.com)  
**Tigi Thomas** (tgthomas@berkeley.edu)  
**Rock Baek** (rockb1017@gmail.com)  
**Hussein Danish** (husseindanish@gmail.com)  
  
Time of Submission: 9:23 PM EST, Wednesday, Feb 17, 2016  
W261-3, Spring 2016  
Week 5 Homework

###Submission Notes:
- For each problem, we've included a summary of the question as posed in the instructions.  In many cases, we have not included the full text to keep the final submission as uncluttered as possible.  For reference, we've included a link to the original instructions in the "Useful Reference" below.
- Problem statements are listed in *italics*, while our responses are shown in plain text. 
- We've included the full output of the mapreduce jobs in our responses so that counter results are shown.  However, these don't always render nicely into PDF form.  In these situations, please reference [the complete rendered notebook on Github](https://github.com/nickhamlin/mids_261_homework/blob/master/HW4/MIDS-W261-2015-HWK-Week04-Hamlin-Thomas-Baek-Danish.ipynb)

###Useful References:
- **[Original Assignment Instructions](https://www.dropbox.com/sh/0cv65h44zylqwe3/AADyEEBMPGezLplMmNwAFIkba/hw5-Questions.txt?dl=0)**

In [1]:
#Use this to make sure we reload the MrJob code when we make changes
%load_ext autoreload
%autoreload 2

##HW5.0.  
*What is a data warehouse? What is a Star schema? When is it used?*

##HW 5.1
*In the database world What is 3NF? Does machine learning use data in 3NF? If so why?*

*In what form does ML consume data?*

*Why would one use log files that are denormalized?*


## HW 5.2

###Problem Statement
Using MRJob, implement a hashside join (memory-backed map-side) for left, 
right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)
:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

(1) Left joining Table Left with Table Right  
(2) Right joining Table Left with Table Right  
(3) Inner joining Table Left with Table Right  

### Generating source data
We'll start by running a slightly modified version of the code from HW4 to generate our two sets of source data

In [3]:
%%writefile convert_msdata.py
#HW 4.2 - Attach customer IDs to page view records

from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='C':
        visitor_id=i[1] #Store visitor id
        continue
    if i[0]=='V':
        print i[0]+','+i[1]+','+i[2]+',C,'+visitor_id #Append visitor_id to each pageview

Writing convert_msdata.py


In [4]:
%%writefile create_urls.py
#HW 4.2 - Extract URLs (not explicitly required, but for later use in 4.4)

#Save only results from 'A' rows into their own file for easy URL access in the future
from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='A':
        print i[1]+','+i[3]+','+i[4]

Writing create_urls.py


In [6]:
%%writefile freq_visitor.py
# HW 4.4 - MRJob Code

import csv
from collections import Counter
from operator import itemgetter

from mrjob.job import MRJob
from mrjob.step import MRStep


def csv_readline(line):
    """Given a string CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

class FreqVisitor(MRJob):

    def mapper_extract_views(self, line_no, line):
        """Extracts the page that was visited and the visitor id"""
        cell = csv_readline(line)
        #Ignore any irrelevant messy data, though hopefully we don't have any since we preprocessed the file
        if cell[0] == 'V': 
            yield cell[1],cell[4]
    
    def reducer_load_urls(self):
        """Load file of page URLs into reducer memory"""
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            #Saving the URLs into a dictionary will make it easy to access them later
            self.url_dict[int(i[0])]=i[2]

    def reducer_sum_views_by_visitor(self, vroots, visitor):
        """Summarizes visitor counts for each page, 
        yields one record per page with the visitor responsible for  
        the most views on that page"""
        visitors=Counter()
        for i in visitor:
            visitors[i]+=1 #Aggregate page views for all visitors
        output= max(visitors.iteritems(), key=itemgetter(1))[0] #Find visitor responsible for the most page views
        yield (str(vroots)),(output,visitors[output],self.url_dict[int(vroots)])
   
    def steps(self):
        return [MRStep(mapper=self.mapper_extract_views,
                        reducer_init=self.reducer_load_urls,
                        reducer=self.reducer_sum_views_by_visitor)]
        
if __name__ == '__main__':
    FreqVisitor.run()

Writing freq_visitor.py


In [ ]:
#Make files executable, convert data, and view some example results to check that everything worked
#!chmod +x convert_msdata.py create_urls.py
!python convert_msdata.py > clean_msdata.txt
!cat clean_msdata.txt | head -10
!python create_urls.py > ms_urls.txt

In [8]:
%%writefile freq_visitor_driver.py
#HW 4.4 - Driver Function
from freq_visitor import FreqVisitor
import csv

mr_job = FreqVisitor(args=['clean_msdata.txt','--file','ms_urls.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        output=mr_job.parse_output_line(line)
        print str(output[0])+'\t'+str(output[1][0])+'\t'+str(output[1][1])+'\t'+str(output[1][2])

Writing freq_visitor_driver.py


In [ ]:
#Make files executable, convert data, and view some example results to check that everything worked
!chmod +x freq_visitor_driver.py
!python freq_visitor_driver.py > freq_visitor_data.txt

### HW 5.2 - Setting up the joins
Using MRJob, implement a hashside join (memory-backed map-side) for left, 

*Justify which table you chose as the Left table in this hashside join.*

Since we're doing a memory-backed map-side join, we want to load the smaller of the two datasets into memory.  Therefore, we'll choose the list of most frequent visitors per page that we generated in 4.4 as our left table and the list of URLs as our right table that we'll load during the mapper_init step.

Please report the number of rows resulting from:

###(1) Left joining Table Left with Table Right   

In [106]:
%%writefile left_join.py
# HW 5.2A - Left join MRJob Code
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class LeftJoin(MRJob):
    
    def mapper_init(self):
        """Load file of page URLs into reducer memory"""
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            #Saving the URLs into a dictionary will make it easy to access them later
            self.url_dict[int(i[0])]=i[2]
        

    def mapper(self, _, line):
        """Extracts the page that was visited and the visitor id"""
        line=line.strip().split('\t')
        page=line[0]
        visitor=line[1]
        #This is the "Left Join" logic that ensures that a row will be returned for
        #every row in the 
        try:
            url=self.url_dict[int(page)]
        except KeyError:
            url='NONE'
        yield page,(visitor,url)
   
    def steps(self):
        return [MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                )]
        
if __name__ == '__main__':
    LeftJoin.run()

Overwriting left_join.py


In [111]:
#HW 5.2 - Left Join Driver Function
from left_join import LeftJoin
import csv

mr_job = LeftJoin(args=['freq_visitor_data.txt','--file','ms_urls.txt'])
number_of_rows=0
with mr_job.make_runner() as runner:
    runner.run()
    #print 'Page | Visitor ID | URL'
    for line in runner.stream_output():
        output=mr_job.parse_output_line(line)
        number_of_rows+=1
        #print output[0], output[1][0], output[1][1]
        
print "Left Join returned {0} results".format(str(number_of_rows))



Page | Visitor ID | URL
Left Join returned 285 results


###(2) Right joining Table Left with Table Right  

In [128]:
%%writefile right_join.py
# HW 5.2A - Left join MRJob Code
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class RightJoin(MRJob):
    
    def mapper_init(self):
        """Load file of page URLs into memory"""
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            #Saving the URLs into a dictionary will make it easy to access them later
            #the second term here is a flag to see if we've emitted a record yet
            self.url_dict[int(i[0])]=[i[2],0] 
        

    def mapper(self, _, line):
        """Extracts the page that was visited and the visitor id"""
        line=line.strip().split('\t')
        page=line[0]
        visitor=line[1]
        #This is the "Inner Join" logic that emits a row for every record appearing in both 
        #tables
        try:
            url=self.url_dict[int(page)][0]
            self.url_dict[int(page)][1]=1 #set flag to indicate we've emitted the record
            yield page,(visitor,url)
        except KeyError:
            pass
        
    
    def mapper_final(self):
        """emit any records in the right table we haven't seen yet"""
        for i in self.url_dict.iteritems():
            if i[1][1]==0:
                page=i[0]
                url=i[1][0]
                yield page,('NONE',url)
   
    def steps(self):
        return [MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper,
                mapper_final=self.mapper_final
                )]
        
if __name__ == '__main__':
    RightJoin.run()

Overwriting right_join.py


In [129]:
#HW 5.2 - Right Join Driver Function
from right_join import RightJoin
import csv

mr_job = RightJoin(args=['freq_visitor_data.txt','--file','ms_urls.txt'])
number_of_rows=0
with mr_job.make_runner() as runner:
    runner.run()
    #print 'Page | Visitor ID | URL'
    for line in runner.stream_output():
        output=mr_job.parse_output_line(line)
        number_of_rows+=1
        #print output[0], output[1][0], output[1][1]
        
print "Right Join returned {0} results".format(str(number_of_rows))



Right Join returned 294 results


###(3) Inner joining Table Left with Table Right 

In [113]:
%%writefile inner_join.py
# HW 5.2A - Inner join MRJob Code
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class InnerJoin(MRJob):
    
    def mapper_init(self):
        """Load file of page URLs into memory"""
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            #Saving the URLs into a dictionary will make it easy to access them later
            #the second term here is a flag to see if we've emitted a record yet
            self.url_dict[int(i[0])]=[i[2],0] 
        

    def mapper(self, _, line):
        """Extracts the page that was visited and the visitor id"""
        line=line.strip().split('\t')
        page=line[0]
        visitor=line[1]
        #This is the "Inner Join" logic that emits a row for every record appearing in both 
        #tables
        try:
            url=self.url_dict[int(page)][0]
            self.url_dict[int(page)][1]=1 #set flag to indicate we've emitted the record
            yield page,(visitor,url)
        except KeyError:
            #Skip records that don't appear in both tables
            pass
        
   
    def steps(self):
        return [MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper
            )]
        
if __name__ == '__main__':
    InnerJoin.run()

Overwriting inner_join.py


In [114]:
#HW 5.2 - Inner Join Driver Function
from inner_join import InnerJoin
import csv

mr_job = InnerJoin(args=['freq_visitor_data.txt','--file','ms_urls.txt'])
number_of_rows=0
with mr_job.make_runner() as runner:
    runner.run()
    #print 'Page | Visitor ID | URL'
    for line in runner.stream_output():
        output=mr_job.parse_output_line(line)
        number_of_rows+=1
        #print output[0], output[1][0], output[1][1]
        
print "Inner Join returned {0} results".format(str(number_of_rows))



Inner Join returned 285 results


## HW 5.3
For the remainder of this assignment you will work with two datasets:

-1: unit/systems test data set: SYSTEMS TEST DATASET
Three terms, A,B,C and their corresponding strip-docs of co-occurring terms

DocA {X:20, Y:30, Z:5}
DocB {X:100, Y:20}
DocC {M:5, N:20, Z:5}


-2: A large subset of the Google n-grams dataset

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket/folder on Dropbox on s3:

   https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACUifrl6wrMrlK6a3X3lZ9Ea?dl=0 

   s3://filtered-5grams/

For each HW 5.3 -5.5 Please unit test and system test your code with with SYSTEMS TEST DATASET and show the results. 
Please compute the expected answer by hand and show your hand calculations. Then show the results you get with you system.
Final show your results on the Google n-grams dataset


In particular, this bucket contains (~200) files (10Meg each) in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

Do some EDA on this dataset using mrjob, e.g., 

- Longest 5-gram (number of characters)
- Top 10 most frequent words (count), i.e., unigrams
- Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency (Hint: save to PART-000* and take the head -n 1000)
- Distribution of 5-gram sizes (counts) sorted in decreasing order of relative frequency. (Hint: save to PART-000* and take the head -n 1000)
- OPTIONAL Question: Plot the log-log plot of the frequency distributuion of unigrams. Does it follow power law distribution?

For more background see:
- https://en.wikipedia.org/wiki/Log%E2%80%93log_plot  
- https://en.wikipedia.org/wiki/Power_law  


### Longest N-Gram (by number of characters)

In [42]:
%%writefile longest_ngram.py
#HW 5.3 - MRJob Definition
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class LongestNgram(MRJob):

    def mapper(self, _, line):
        """ """
        line=line.strip().split('\t')
        ngram=line[0]
        yield None,(len(ngram),ngram)
        
    def reducer(self, _, ngram_and_length):
        """ """
        yield None, max(ngram_and_length)
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper
                   ,combiner=self.reducer
                    ,reducer=self.reducer
                  )        
        ]
        
if __name__ == '__main__':
    LongestNgram.run()

Overwriting longest_ngram.py


In [43]:
#HW 4.3 - Driver Function
from longest_ngram import LongestNgram
#import csv

mr_job = LongestNgram(args=['testngrams.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)


(None, [34, 'A HANDBOOK ON THEODOLITE SURVEYING'])


### Top 10 Most Frequent Words

In [6]:
%%writefile most_freq_words.py
#HW 5.3 - MRJob Definition
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class MostFreqWords(MRJob):
    
    def jobconf(self):
        orig_jobconf = super(TopPages, self).jobconf()        
        custom_jobconf = {  #key value pairs
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2nr',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

    def mapper(self, _, line):
        """ """
        line=line.strip().split('\t')
        ngram=line[0]
        for word in ngram.split(' '):
            yield word,1
            
    def reducer(self,word,count):
        yield (word,sum(count)),None
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper
                   #,combiner=self.reducer
                    ,reducer=self.reducer
                  )        
        ]
        
if __name__ == '__main__':
    MostFreqWords.run()

Overwriting most_freq_words.py


In [7]:
#HW 4.3 - Driver Function
from most_freq_words import MostFreqWords

mr_job = MostFreqWords(args=['testngrams.txt','-r','local'])#,'-hadoop_home',"/Users/nicholashamlin/hadoop-2.6.3"])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)


ERROR:mrjob.local:STDERR: + __mrjob_PWD=/private/var/folders/rz/drh189k95919thyy3gs3tq400000gn/T/most_freq_words.nicholashamlin.20160214.024655.594496/job_local_dir/0/mapper/0
ERROR:mrjob.local:STDERR: + exec
ERROR:mrjob.local:STDERR: + /Users/nicholashamlin/anaconda/bin/python -c 'import fcntl; fcntl.flock(9, fcntl.LOCK_EX)'
ERROR:mrjob.local:STDERR: + export PYTHONPATH=/private/var/folders/rz/drh189k95919thyy3gs3tq400000gn/T/most_freq_words.nicholashamlin.20160214.024655.594496/job_local_dir/0/mapper/0/mrjob.tar.gz:
ERROR:mrjob.local:STDERR: + PYTHONPATH=/private/var/folders/rz/drh189k95919thyy3gs3tq400000gn/T/most_freq_words.nicholashamlin.20160214.024655.594496/job_local_dir/0/mapper/0/mrjob.tar.gz:
ERROR:mrjob.local:STDERR: + exec
ERROR:mrjob.local:STDERR: + cd /private/var/folders/rz/drh189k95919thyy3gs3tq400000gn/T/most_freq_words.nicholashamlin.20160214.024655.594496/job_local_dir/0/mapper/0
ERROR:mrjob.local:STDERR: + /Users/nicholashamlin/anaconda/bin/python most_freq_words.p

(['A', 50], None)
(['ALL', 1], None)
(['AT', 1], None)
(['Aerial', 1], None)
(["America's", 1], None)
(['American', 1], None)
(['Apology', 1], None)
(['Arithmetic', 1], None)
(['BILL', 1], None)
(['Bibliographical', 1], None)
(['Bibliography', 1], None)
(['Biography', 1], None)
(['Case', 3], None)
(["Cat's", 1], None)
(['Censorship', 1], None)
(["Child's", 1], None)
(['Christmas', 1], None)
(['Circumstantial', 1], None)
(['City', 1], None)
(['Collection', 2], None)
(['Commentary', 1], None)
(['Comparative', 1], None)
(['Comparison', 1], None)
(['Conceptual', 2], None)
(['Concise', 1], None)
(['Continuation', 1], None)
(['Cradle', 1], None)
(['Critical', 1], None)
(['Critique', 1], None)
(['Defence', 2], None)
(['Dirty', 1], None)
(['Discovery', 1], None)
(['ESTABLISHING', 1], None)
(['Eurobond', 1], None)
(['FOR', 3], None)
(['FURTHER', 1], None)
(['Fairy', 1], None)
(['Female', 1], None)
(['Festschrift', 1], None)
(['Forms', 1], None)
(['Framework', 2], None)
(['Funny', 1], None)
(['G

#TODO:
- Figure out where this data lives in S3
- Put test data in S3
- Add second stage job to this that uses identity mappers/reducers and sorts with jobconf
- Test job on AWS
- Run full job on AWS (How big a cluster do you recommend for this?)

### Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency (Hint: save to PART-000* and take the head -n 1000)

In [162]:
%%writefile word_density.py
#HW 5.3 - MRJob Definition
from __future__ import division
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class WordDensity(MRJob):

    def mapper(self, _, line):
        """ """
        line=line.strip().split('\t')
        ngram=line[0]
        count=line[1]
        page_count=line[2]
        for word in ngram.split(' '):
            yield word,(count,page_count)
            
    def combiner(self,word,count):
        word_count=0
        page_count=0
        for record in count:
            word_count+=int(record[0])
            page_count+=int(record[1])
        yield word,(word_count,page_count)
                    
    def reducer(self,word,count):
        word_count=0
        page_count=0
        for record in count:
            word_count+=int(record[0])
            page_count+=int(record[1])
        yield word,word_count/page_count
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper
                   ,combiner=self.combiner
                    ,reducer=self.reducer
                  )        
        ]
        
if __name__ == '__main__':
    WordDensity.run()

Overwriting word_density.py


In [163]:
#HW 4.3 - Driver Function
from word_density import WordDensity

mr_job = WordDensity(args=['testngrams.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)


('A', 1.0588044078925982)
('ALL', 1.0301369863013699)
('AT', 1.02)
('Aerial', 1.0)
("America's", 1.0)
('American', 1.0058479532163742)
('Apology', 1.0)
('Arithmetic', 1.0)
('BILL', 1.0)
('Bibliographical', 1.0)
('Bibliography', 1.013986013986014)
('Biography', 1.0222222222222221)
('Case', 1.0)
("Cat's", 1.0)
('Censorship', 1.0)
("Child's", 1.0358152686145146)
('Christmas', 1.0358152686145146)
('Circumstantial', 1.0)
('City', 1.0333333333333334)
('Collection', 1.0863636363636364)
('Commentary', 1.0)
('Comparative', 1.0625)
('Comparison', 1.0)
('Conceptual', 1.0)
('Concise', 1.013986013986014)
('Continuation', 1.0196078431372548)
('Cradle', 1.0)
('Critical', 1.0154639175257731)
('Critique', 1.0)
('Defence', 1.028423772609819)
('Dirty', 1.1688311688311688)
('Discovery', 1.0079681274900398)
('ESTABLISHING', 1.0)
('Eurobond', 1.0172413793103448)
('FOR', 1.0441767068273093)
('FURTHER', 1.02)
('Fairy', 1.0512820512820513)
('Female', 1.0)
('Festschrift', 1.0166666666666666)
('Forms', 1.1262135

### Distribution of 5-gram sizes (counts) sorted in decreasing order of relative frequency. (Hint: save to PART-000* and take the head -n 1000)

#TODO:
- I assume here we use the count attribute for each row, calculate a relative frequency of those counts

In [96]:
%%writefile ngram_distribution.py
#HW 5.3 - MRJob Definition
from __future__ import division
import csv

from mrjob.job import MRJob
from mrjob.step import MRStep

class NgramDistribution(MRJob):
    
    def mapper_init(self):
        self.count=0

    def mapper(self, _, line):
        """ """
        line=line.strip().split('\t')
        ngram=line[0]
        ngram_count=int(line[1])
        self.count+=ngram_count
        yield ngram,ngram_count
    
    def mapper_final(self):
        yield '*count',self.count
        
    def reducer_init(self):
        self.total_count=0
            
    def reducer(self,ngram,ngram_count):
        print ngram,sum(ngram_count)
        if ngram=='*count':
            print 'ok'
            self.total_count+=sum(ngram_count)
        print self.total_count
        yield ngram,(sum(ngram_count))/self.total_count
        
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init,
                mapper=self.mapper
                ,mapper_final=self.mapper_final
                #,combiner=self.reducer
                ,reducer_init=self.reducer_init
                ,reducer=self.reducer
                  )        
        ]
        
if __name__ == '__main__':
    NgramDistribution.run()

Overwriting ngram_distribution.py


In [97]:
#HW 4.3 - Driver Function
from ngram_distribution import NgramDistribution

mr_job = NgramDistribution(args=['testngrams.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)


*count 32572
ok
0


ZeroDivisionError: division by zero

## HW 5.4

###Problem Statement
In this part of the assignment we will focus on developing methods
for detecting synonyms, using the Google 5-grams dataset. To accomplish
this you must script two main tasks using MRJob:

(1) Build stripes of word co-ocurrence for the top 10,000 using the words ranked from 9001,-10,000 as a basis
most frequently appearing words across the entire set of 5-grams,
and output to a file in your bucket on s3 (bigram analysis, though the words are non-contiguous).

(2) Using two (symmetric) comparison methods of your choice 
(e.g., correlations, distances, similarities), pairwise compare 
all stripes (vectors), and output to a file in your bucket on s3.

==Design notes for (1)==  
For this task you will be able to modify the pattern we used in HW 3.2
(feel free to use the solution as reference). To total the word counts 
across the 5-grams, output the support from the mappers using the total 
order inversion pattern:

<*word,count>

to ensure that the support arrives before the cooccurrences.

In addition to ensuring the determination of the total word counts,
the mapper must also output co-occurrence counts for the pairs of
words inside of each 5-gram. Treat these words as a basket,
as we have in HW 3, but count all stripes or pairs in both orders,
i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

==Design notes for (2)==  
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

- Jaccard
- Cosine similarity
- Spearman correlation
- Euclidean distance
- Taxicab (Manhattan) distance
- Shortest path graph distance (a graph, because our data is symmetric!)
- Pearson correlation
- Kendall correlation
...

However, be cautioned that some comparison methods are more difficult to
parallelize than others, and do not perform more associations than is necessary, 
since your choice of association will be symmetric.

Please use the inverted index (discussed in live session #5) based pattern to compute the pairwise (term-by-term) similarity matrix. 

## HW 5.5

###Problem Statement
In this part of the assignment you will evaluate the success of you synonym detector.
Take the top 1,000 closest/most similar/correlative pairs of words as determined
by your measure in (2), and use the synonyms function in the accompanying
python code:

nltk_synonyms.py

For each (word1,word2) pair, check to see if word1 is in the list, 
synonyms(word2), and vice-versa. If one of the two is a synonym of the other, 
then consider this pair a 'hit', and then report the precision, recall, and F1 measure  of 
your detector across your 1,000 best guesses. Report the macro averages of these measures.

In [2]:
#!/usr/bin/python2.7
''' pass a string to this funciton ( eg 'car') and it will give you a list of
words which is related to cat, called lemma of CAT. '''
import nltk
from nltk.corpus import wordnet as wn
import sys
#print all the synset element of an element
def synonyms(string):
    syndict = {}
    for i,j in enumerate(wn.synsets(string)):
        syns = j.lemma_names()
        for syn in syns:
            syndict.setdefault(syn,1)
    return syndict.keys()


###End of Submission